In [13]:
# imports
import pandas as pd
import requests
import os
from pprint import pprint


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
%store -r cb_lat_long
#api_key = os.environ["foursquare_api"] 
#url = "https://api.foursquare.com/v3/places/search?near=" + location
#headers = {"Accept": "application/json"}
#headers['Authorization'] = api_key
#result = requests.get(url, headers=headers)
# need a for loop to cycle through list of lat/long, must be in string, radius is constant 1000
for latlong in cb_lat_long:
    location = latlong
print(location)
#pprint(cb_lat_long)

[49.270783, -123.141564]


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating